In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from urllib import urlretrieve
import cPickle as pickle
import os
import gzip
import numpy as np
import theano
import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Load Cifar-10

In [4]:
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [3]:
batch1=unpickle('cifar-10-batches-py/data_batch_1')
batch2=unpickle('cifar-10-batches-py/data_batch_2')
batch3=unpickle('cifar-10-batches-py/data_batch_3')
batch4=unpickle('cifar-10-batches-py/data_batch_4')
batch5=unpickle('cifar-10-batches-py/data_batch_5')
testbatch=unpickle('cifar-10-batches-py/test_batch')

In [4]:
X_train=np.concatenate((batch1[b'data'],batch2[b'data'],batch3[b'data'],batch4[b'data'],batch5[b'data']),axis=0)
y_train=np.concatenate((batch1[b'labels'],batch2[b'labels'],batch3[b'labels'],batch4[b'labels'],batch5[b'labels']),axis=0)
#X_val, y_val, 
X_test=testbatch[b'data']
y_test=testbatch[b'labels']

In [5]:
labelname=unpickle('cifar-10-batches-py/batches.meta')

In [7]:
labelname['label_names']

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

X_train -= np.mean(X_train, axis = 0).astype(X_train.dtype) # zero-center
X_train /= np.std(X_train, axis = 0).astype(X_train.dtype)

def whiten(X,fudge=1E-18):

   # the matrix X should be observations-by-components

   # get the covariance matrix
   Xcov = np.dot(X.T,X)

   # eigenvalue decomposition of the covariance matrix
   d, V = np.linalg.eigh(Xcov)

   # a fudge factor can be used so that eigenvectors associated with
   # small eigenvalues do not get overamplified.
   D = np.diag(1. / np.sqrt(d+fudge))

   # whitening matrix
   W = np.dot(np.dot(V, D), V.T)

   # multiply by the whitening matrix
   X_white = np.dot(X, W)

   return X_white, W


def svd_whiten(X):

    U, s, Vt = np.linalg.svd(X)

    # U and Vt are the singular matrices, and s contains the singular values.
    # Since the rows of both U and Vt are orthonormal vectors, then U * Vt
    # will be white
    X_white = np.dot(U, Vt)

    return X_white


X_pca=[]
for x in new_X_train:
    x=whiten(x)
    X_pca.append(x)

X_svd=[]
for x in new_X_train:
    x=svd_whiten(x)
    X_svd.append(x)

cov = np.dot(X_train.T, X_train) / X_train.shape[0]

## CNN without Data Preprocessing

In [15]:
X_train1=np.array(X_train)
X_train1=np.reshape(X_train, (-1,3,32,32), order='F')
y_train=np.array(y_train)
y_train= y_train.astype(np.int32) 

In [16]:
net1 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            ('maxpool2', layers.MaxPool2DLayer),
            #('conv2d3', layers.Conv2DLayer),
            #('maxpool3', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 3, 32, 32),
    # layer conv2d1
    conv2d1_num_filters=32,
    conv2d1_filter_size=(3, 3),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(2, 2),    
    # layer conv2d2
    conv2d2_num_filters=64,
    conv2d2_filter_size=(3, 3),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool2
    maxpool2_pool_size=(2, 2),
    # layer conv2d3
    #conv2d3_num_filters=128,
    #conv2d3_filter_size=(3, 3),
    #conv2d3_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool3
    #maxpool3_pool_size=(2, 2),
    # dropout1
    dropout1_p=0.5,    
    # dense
    dense_num_units=256,
    dense_nonlinearity=lasagne.nonlinearities.rectify,    
    # dropout2
    dropout2_p=0.5,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=10,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,
    max_epochs=10,
    verbose=1,
    )

In [17]:
nn1 = net1.fit(X_train1, y_train)

# Neural Network with 612042 learnable parameters

## Layer information

  #  name      size
---  --------  --------
  0  input     3x32x32
  1  conv2d1   32x30x30
  2  maxpool1  32x15x15
  3  conv2d2   64x13x13
  4  maxpool2  64x6x6
  5  dropout1  64x6x6
  6  dense     256
  7  dropout2  256
  8  output    10

  epoch     trn loss    val loss      trn/val    valid acc  dur
-------  -----------  ----------  -----------  -----------  ------
      1  89376.60280     2.30269  38813.96232      0.10000  98.04s
      2     2.30293     2.30267    1.00011      0.10000  104.02s
      3     2.30292     2.30267    1.00011      0.10000  102.03s
      4     2.30292     2.30267    1.00011      0.10000  102.73s
      5     2.30292     2.30267    1.00011      0.10000  105.33s
      6     2.30292     2.30267    1.00011      0.10000  109.48s
      7     2.30292     2.30267    1.00011      0.10000  102.87s
      8     2.30292     2.30267    1.00011      0.10000  105.37s
      9     2.30292     2.30267   

In [22]:
X_train_rescaling=X_train1/255

In [23]:
net2 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            ('maxpool2', layers.MaxPool2DLayer),
            #('conv2d3', layers.Conv2DLayer),
            #('maxpool3', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 3, 32, 32),
    # layer conv2d1
    conv2d1_num_filters=32,
    conv2d1_filter_size=(3, 3),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(2, 2),    
    # layer conv2d2
    conv2d2_num_filters=64,
    conv2d2_filter_size=(3, 3),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool2
    maxpool2_pool_size=(2, 2),
    # layer conv2d3
    #conv2d3_num_filters=96,
    #conv2d3_filter_size=(3, 3),
    #conv2d3_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool3
    #maxpool3_pool_size=(2, 2),
    # dropout1
    dropout1_p=0.5,    
    # dense
    dense_num_units=256,
    dense_nonlinearity=lasagne.nonlinearities.rectify,    
    # dropout2
    dropout2_p=0.5,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=10,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,
    max_epochs=10,
    verbose=1,
    )

In [24]:
nn2 = net2.fit(X_train_rescaling, y_train)

# Neural Network with 612042 learnable parameters

## Layer information

  #  name      size
---  --------  --------
  0  input     3x32x32
  1  conv2d1   32x30x30
  2  maxpool1  32x15x15
  3  conv2d2   64x13x13
  4  maxpool2  64x6x6
  5  dropout1  64x6x6
  6  dense     256
  7  dropout2  256
  8  output    10

  epoch    trn loss    val loss    trn/val    valid acc  dur
-------  ----------  ----------  ---------  -----------  -------
      1     2.28813     2.26828    1.00875      0.13230  100.13s
      2     2.26901     2.25030    1.00831      0.13870  96.89s
      3     2.25857     2.23897    1.00876      0.15830  96.29s
      4     2.25045     2.23178    1.00837      0.16170  96.52s
      5     2.24470     2.22631    1.00826      0.16420  97.27s
      6     2.24052     2.22292    1.00792      0.16570  96.43s
      7     2.23681     2.21985    1.00764      0.16440  96.86s
      8     2.23351     2.21763    1.00716      0.16600  96.54s
      9     2.23059     2.21570    1.00672      

# Data Preprocessing
#### RGB to single grayscale
#### Flatten grayscale
#### ZCA_Whitening/PCA_Whitening

In [8]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def flatten_matrix(matrix):
    vector = matrix.flatten(1)
    vector = vector.reshape(1, len(vector))
    return vector    

In [9]:
new_X_train=[]
for img in X_train:
    img=np.reshape(img, (32,32,3), order='F')
    gray=flatten_matrix(rgb2gray(img)/255)
    new_X_train.append(gray)

/Users/Shuxian/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


In [10]:
def zca_whitening(inputs):
    sigma = np.dot(inputs, inputs.T)/inputs.shape[1] #Correlation matrix
    U,S,V = np.linalg.svd(sigma) #Singular Value Decomposition
    epsilon = 0.1                #Whitening constant, it prevents division by zero
    ZCAMatrix = np.dot(np.dot(U, np.diag(1.0/np.sqrt(np.diag(S) + epsilon))), U.T) #ZCA Whitening matrix
    return np.dot(ZCAMatrix, inputs)   #Data whitening

In [11]:
new_X_train=np.asarray(new_X_train)
X_zca=[]
for x in new_X_train:
    x=zca_whitening(x)
    X_zca.append(x)

In [12]:
X_zca=np.array(X_zca)
X_zca=np.reshape(X_zca,(-1,1,32,32))

In [11]:
img=np.reshape(X_train[0], (32,32,3), order='F')
gray = rgb2gray(img)/255
#img = mpimg.imread('image.png')
plt.subplot(1, 2, 1); 
plt.axis('off');
plt.imshow(img);

plt.subplot(1, 2, 2); 
plt.axis('off');
plt.imshow(gray, cmap = plt.get_cmap('gray'));

#plt.subplot(1, 3, 3); 
#plt.axis('off');
#plt.imshow(X_zca[0],cmap=cm.binary);

plt.show()

In [13]:
net3 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            ('maxpool2', layers.MaxPool2DLayer),
            # ('conv2d3', layers.Conv2DLayer),
            # ('maxpool3', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 1, 32, 32),
    # layer conv2d1
    conv2d1_num_filters=32,
    conv2d1_filter_size=(3, 3),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(2, 2),    
    # layer conv2d2
    conv2d2_num_filters=64,
    conv2d2_filter_size=(3, 3),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool2
    maxpool2_pool_size=(2, 2),
    # layer conv2d3
    # conv2d3_num_filters=96,
    # conv2d3_filter_size=(3, 3),
    # conv2d3_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool3
    # maxpool3_pool_size=(2, 2),
    # dropout1
    dropout1_p=0.2,    
    # dense
    dense_num_units=256,
    dense_nonlinearity=lasagne.nonlinearities.rectify,    
    # dropout2
    dropout2_p=0.2,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=10,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.005,
    update_momentum=0.9,
    max_epochs=15,
    verbose=1,
    )

In [16]:
# Train the network
nn3 = net3.fit(X_zca, y_train)

# Neural Network with 611466 learnable parameters

## Layer information

  #  name      size
---  --------  --------
  0  input     1x32x32
  1  conv2d1   32x30x30
  2  maxpool1  32x15x15
  3  conv2d2   64x13x13
  4  maxpool2  64x6x6
  5  dropout1  64x6x6
  6  dense     256
  7  dropout2  256
  8  output    10

  epoch    trn loss    val loss    trn/val    valid acc  dur
-------  ----------  ----------  ---------  -----------  ------
      1     2.20571     1.98018    1.11389      0.30890  99.63s
      2     1.84449     1.69058    1.09104      0.40880  99.41s
      3     1.67439     1.56577    1.06937      0.45280  98.41s
      4     1.58314     1.48115    1.06886      0.48240  97.75s
      5     1.50145     1.42872    1.05091      0.49940  100.27s
      6     1.43279     1.34642    1.06415      0.52900  99.20s
      7     1.36316     1.30511    1.04449      0.54750  99.57s
      8     1.30864     1.26003    1.03858      0.56580  96.16s
      9     1.25544     1.20570    1.04126      0

In [17]:
new_X_test=[]
for img in X_test:
    img=np.reshape(img, (32,32,3), order='F')
    gray=flatten_matrix(rgb2gray(img)/255)
    gray=np.asarray(gray)
    x=zca_whitening(gray)
    new_X_test.append(x)

new_X_test=np.array(new_X_test)
new_X_test=np.reshape(new_X_test,(-1,1,32,32))
y_test=np.array(y_test)
y_test= y_test.astype(np.int32)

/Users/Shuxian/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


In [18]:
preds = net3.predict(new_X_test)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

0.62780000000000002

In [42]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds)

1.2853000000000001

In [19]:
cm = confusion_matrix(y_test, preds)
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

/Users/Shuxian/anaconda/envs/py27/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [22]:
visualize.plot_conv_weights(net3.layers_['conv2d1'])
plt.show()

/Users/Shuxian/anaconda/envs/py27/lib/python2.7/site-packages/matplotlib/image.py:676: UserWarning: The backend (<class 'matplotlib.backends.backend_macosx.RendererMac'>) does not support interpolation='none'. The image will be interpolated with 'nearest` mode.
  "mode." % renderer.__class__)


In [24]:
dense_layer = layers.get_output(net3.layers_['dense'], deterministic=True)
output_layer = layers.get_output(net3.layers_['output'], deterministic=True)
input_var = net3.layers_['input'].input_var
f_output = theano.function([input_var], output_layer)
f_dense = theano.function([input_var], dense_layer)

In [39]:
instance = X_test[0][None, :,:]
%timeit -n 500 f_output(instance)

#500 loops, best of 3: 858 µs per loop

IndexError: too many indices for array

In [ ]:
pred = f_output(instance)
N = pred.shape[1]
plt.bar(range(N), pred.ravel())
plt.show()

In [ ]:
pred = f_dense(instance)
N = pred.shape[1]
plt.bar(range(N), pred.ravel())
plt.show()

In [35]:
X_test[0].shape

(3072,)